<a href="https://colab.research.google.com/github/aaptecode/DS-Unit-4-Sprint-2-NLP/blob/master/LS_DS_424_Word_Embeddings_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [11]:
##### Your Code Here #####

!pip install -U gensim
import gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.7.1)


In [12]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [0]:
from google.colab import files

import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
import warnings

style.use('seaborn-whitegrid')
warnings.filterwarnings('ignore')

## Data

### Data fields

**id** - the id of a training set question pair

**qid1, qid2** - unique ids of each question (only available in train.csv)

**question1, question2** - the full text of each question

**is_duplicate** - the target variable, set to 1 if question1 and question2 have essentially the same meaning, and 0 otherwise.


In [14]:
uploaded = files.upload()

Saving train.csv to train (1).csv


In [15]:
df = pd.read_csv('train.csv')

print(df.shape)
df.head()

(404290, 6)


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [18]:
df['is_duplicate'].value_counts(normalize = True)

0    0.630802
1    0.369198
Name: is_duplicate, dtype: float64

**Cleaning Data and Tokenizing Documents**

In [16]:
from nltk.corpus import stopwords
import string

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

df['cleaned1'] = df.question1.astype(str).apply(clean_doc)
df['cleaned2'] = df.question2.astype(str).apply(clean_doc)
print(df.shape)
df.head()

(404290, 8)


,id,qid1,qid2,question1,question2,is_duplicate,cleaned1,cleaned2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[What, step, step, guide, invest, share, marke...","[What, step, step, guide, invest, share, market]"
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[What, story, Kohinoor, KohiNoor, Diamond]","[What, would, happen, Indian, government, stol..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[How, increase, speed, internet, connection, u...","[How, Internet, speed, increased, hacking, DNS]"
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[Why, mentally, lonely, How, solve]","[Find, remainder, divided]"
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[Which, one, dissolve, water, quikly, sugar, s...","[Which, fish, would, survive, salt, water]"


In [19]:
final_list_q1 = [' '.join(document) for document in df.cleaned1]
final_list_q2 = [' '.join(document) for document in df.cleaned2]
# for document in simpsons.cleaned:
#     final_list.append(' '.join(document))
    
    
  
final1 = pd.Series(final_list_q1)
final2 = pd.Series(final_list_q2)
df['final1'] = final1
df['final2'] = final2
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,cleaned1,cleaned2,final1,final2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,"[What, step, step, guide, invest, share, marke...","[What, step, step, guide, invest, share, market]",What step step guide invest share market india,What step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,"[What, story, Kohinoor, KohiNoor, Diamond]","[What, would, happen, Indian, government, stol...",What story Kohinoor KohiNoor Diamond,What would happen Indian government stole Kohi...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,"[How, increase, speed, internet, connection, u...","[How, Internet, speed, increased, hacking, DNS]",How increase speed internet connection using VPN,How Internet speed increased hacking DNS
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,"[Why, mentally, lonely, How, solve]","[Find, remainder, divided]",Why mentally lonely How solve,Find remainder divided
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"[Which, one, dissolve, water, quikly, sugar, s...","[Which, fish, would, survive, salt, water]",Which one dissolve water quikly sugar salt met...,Which fish would survive salt water


In [21]:
print(df.cleaned1[:10]) 
print()
print(df.cleaned2[:10])

0    [What, step, step, guide, invest, share, marke...
1           [What, story, Kohinoor, KohiNoor, Diamond]
2    [How, increase, speed, internet, connection, u...
3                  [Why, mentally, lonely, How, solve]
4    [Which, one, dissolve, water, quikly, sugar, s...
5    [Astrology, Capricorn, Sun, Cap, moon, cap, ri...
6                                 [Should, buy, tiago]
7                               [How, good, geologist]
8                                 [When, use, instead]
9    [Motorola, company, Can, hack, Charter, Motoro...
Name: cleaned1, dtype: object

0     [What, step, step, guide, invest, share, market]
1    [What, would, happen, Indian, government, stol...
2      [How, Internet, speed, increased, hacking, DNS]
3                           [Find, remainder, divided]
4           [Which, fish, would, survive, salt, water]
5    [Im, triple, Capricorn, Sun, Moon, ascendant, ...
6    [What, keeps, childern, active, far, phone, vi...
7                             [Wha

In [0]:
%%time

from gensim.models import Word2Vec
w2v_1 = Word2Vec(df.cleaned1, min_count=20, window=3, size=300) #, negative=20)
w2v_2 = Word2Vec(df.cleaned2, min_count=20, window=3, size=300) #, negative=20)

In [26]:
words1 = list(w2v_1.wv.vocab)
words2 = list(w2v_2.wv.vocab)
print(f'Vocabulary Size of 1: {len(words1)}')
print(f'Vocabulary Size of 2: {len(words2)}')

Vocabulary Size of 1: 10327
Vocabulary Size of 2: 10129


**most_similar()**

In [36]:
w2v_1.wv.most_similar('india', topn=10) # Default is positive

[('India', 0.7353679537773132),
 ('INDIA', 0.6909715533256531),
 ('Australia', 0.6725416779518127),
 ('opportunity', 0.6566749215126038),
 ('funds', 0.648633599281311),
 ('internationally', 0.6432605981826782),
 ('investments', 0.6407665014266968),
 ('deals', 0.6401877999305725),
 ('Dubai', 0.6384397745132446),
 ('USA', 0.6277740001678467)]

In [38]:
w2v_1.wv.most_similar(negative=['india'], topn=10)

[('word', 0.41762346029281616),
 ('personality', 0.38193267583847046),
 ('hear', 0.38063859939575195),
 ('words', 0.37739813327789307),
 ('phrase', 0.35149645805358887),
 ('disorder', 0.3421269655227661),
 ('translate', 0.3418766260147095),
 ('translation', 0.3349642753601074),
 ('signs', 0.32094499468803406),
 ('symbol', 0.3142086863517761)]

In [37]:
w2v_2.wv.most_similar('india', topn=10) # Default is positive

[('India', 0.7996655702590942),
 ('INDIA', 0.6601487994194031),
 ('UAE', 0.6383296847343445),
 ('infrastructure', 0.6371738910675049),
 ('overseas', 0.6307905912399292),
 ('considering', 0.6165410280227661),
 ('offering', 0.6109635829925537),
 ('sector', 0.6076132655143738),
 ('USA', 0.6041314601898193),
 ('Dubai', 0.5984183549880981)]

In [39]:
w2v_2.wv.most_similar(negative=['india'], topn=10)

[('word', 0.37535524368286133),
 ('bipolar', 0.35107421875),
 ('words', 0.3483104407787323),
 ('disorder', 0.34806931018829346),
 ('personality', 0.34219595789909363),
 ('signs', 0.33505114912986755),
 ('accurately', 0.3193957805633545),
 ('rare', 0.31840452551841736),
 ('shape', 0.3182941973209381),
 ('symptoms', 0.31567978858947754)]

**similarity()**

In [40]:
w2v_1.wv.similarity("india", "china")

0.41441512

In [41]:
w2v_2.wv.similarity("india", "china")

0.447466

**doesnt_match()**

In [46]:
w2v_1.wv.doesnt_match(['india', 'pakistan', 'china'])

'india'

In [45]:
w2v_2.wv.doesnt_match(['india', 'pakistan', 'china'])

'india'

**Analogies with most_similar()**

In [47]:
w2v_1.wv.most_similar(positive=["india", "pakistan"], negative=["china"], topn=3)

[('India', 0.640143871307373),
 ('Kashmir', 0.5330369472503662),
 ('Pakistan', 0.5283345580101013)]

In [48]:
w2v_2.wv.most_similar(positive=["india", "pakistan"], negative=["china"], topn=3)

[('India', 0.7277836203575134),
 ('fund', 0.5583522319793701),
 ('hedge', 0.543509840965271)]

### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

Load the pre-trained word vectors and train the Word2vec model

Examine the first 100 keys or words of the vocabulary

Outputs the vector representation for a select set of words - the words can be of your choice

Examine the similarity between words - the words can be of your choice

For example:

model.similarity('house', 'bungalow')

model.similarity('house', 'umbrella')